In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Database Credentials
host = 'localhost'
port = '5432'
database = 'apple_sales_db'
user = 'postgres'
password = '1234'

# Create Connection
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Test Connection
try:
    with engine.connect() as conn:
        print("✓ Connection successful!")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    exit()

# File to Table Mapping
files_map = {
    'category.csv': 'category',
    'products.csv': 'products',
    'stores.csv': 'stores',
    'warranty.csv': 'warranty',
    'sales.csv': 'sales'
}

data_folder = './data'

# Upload Process
print("Starting upload process...")

for file, table in files_map.items():
    file_path = os.path.join(data_folder, file)
    
    if os.path.exists(file_path):
        try:
            print(f"Processing {file}...")
            df = pd.read_csv(file_path)
            df.columns = [c.lower() for c in df.columns]
            
            # Uploading with chunksize to handle large files safely
            df.to_sql(name=table, con=engine, if_exists='replace', index=False, chunksize=10000)
            print(f"✓ {table} uploaded.")
        except Exception as e:
            print(f"✗ Error uploading {file}: {e}")
    else:
        print(f"⚠ File not found: {file_path}")

print("All operations completed.")